In [24]:
import pandas as pd 

In [25]:
df = pd.read_csv("./data/combined_twitter_data.csv")

In [26]:
df_copy = df.copy()

In [27]:
df_copy.head()

,Unnamed: 0,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,...,profile_background_image_url,profile_background_color,profile_link_color,utc_offset,protected,verified,description,created_at,updated,account_type
0,1,22903,effeffe,effeffe,164,132,194,12,4,http://t.co/CX7EHdk9DJ,...,http://a0.twimg.com/images/themes/theme14/bg.gif,131516,9999,3600.0,NaN,NaN,L'uomo ha creato dio a sua immagine e somiglia...,Sun Nov 26 15:19:32 +0000 2006,14/2/2015 11:32,real
1,3,286543,Alessio Bragadini,abragad,6892,930,535,478,28,http://t.co/xoOdZi9jic,...,http://a0.twimg.com/profile_background_images/...,9AE4E8,0000FF,3600.0,NaN,NaN,Web and social media developer from Italy,Wed Dec 27 14:55:17 +0000 2006,14/2/2015 11:32,real
2,4,438023,fullcaffeine,fullcaffeine,2885,173,444,41,2,http://www.fullcaffeine.com,...,http://a0.twimg.com/images/themes/theme14/bg.gif,131516,9999,3600.0,NaN,NaN,NaN,Tue Jan 02 09:01:50 +0000 2007,14/2/2015 11:32,real
3,5,586003,Maurizio Tesconi,myself2048,216,97,234,2,0,http://maurizio.tesconi.it/,...,http://a0.twimg.com/images/themes/theme1/bg.png,9AE4E8,0000FF,3600.0,NaN,NaN,CNR Researcher. Web developer. Social media ge...,Fri Jan 05 16:20:42 +0000 2007,14/2/2015 11:32,real
4,6,628563,Massimo Moretti,MaxMoretti,505,154,314,0,3,http://www.moretti.rm.it/,...,http://a0.twimg.com/profile_background_images/...,C0DEED,0084B4,3600.0,NaN,NaN,The truth is out there!,Fri Jan 12 11:06:17 +0000 2007,14/2/2015 11:32,real


In [28]:
df_copy.columns

Index(['Unnamed: 0', 'id', 'name', 'screen_name', 'statuses_count',
       'followers_count', 'friends_count', 'favourites_count', 'listed_count',
       'url', 'lang', 'time_zone', 'location', 'default_profile',
       'default_profile_image', 'geo_enabled', 'profile_image_url',
       'profile_banner_url', 'profile_use_background_image',
       'profile_background_image_url_https', 'profile_text_color',
       'profile_image_url_https', 'profile_sidebar_border_color',
       'profile_background_tile', 'profile_sidebar_fill_color',
       'profile_background_image_url', 'profile_background_color',
       'profile_link_color', 'utc_offset', 'protected', 'verified',
       'description', 'created_at', 'updated', 'account_type'],
      dtype='object')

In [29]:
def create_followers_following_ratio(df):
    df['following_to_followers_ratio'] = df['friends_count'] / df['followers_count']
    return df

In [30]:
def has_url_feature(df):
    df['has_url'] = df['url'].isna()
    df['has_url'] = df['has_url'].apply(lambda x: 0 if x == True else 1)
    return df

#2685 has url, 8427 has no url (is na)

In [77]:
df_tweets = pd.read_csv("./data/combined_twitter_data_with_tweets_corpus.csv")

str

In [128]:
str(df_tweets.tweets_list[200])

'[\'RT @securityaffairs: Dissecting a mobile malwarehttp://t.co/sr4cAUV5#malware #cybercrime #banking #mobile #Android\', \'Inscrito Ã¢â\\x82¬Å\\x93@S21sec: Recordatorio, Seminario online gratuito: "Webmalware, una nueva evoluciÃ\\x83Â³n del cibercrimen"http://t.co/kxiGwiGuÃ¢â\\x82¬Â\\x9d\', \'RT @bquintero: nuevos juegos para el spectrum en 2012 http://t.co/rqCyojL2 buena coleccion aquÃ\\x83Â\\xad http://t.co/YlF6uNQs\', \'Differential Privacy for Everyone - a non-technical introduction to differential privacy by Microsoft http://t.co/SV2cVSd6\', \'RT @secbydefault: [SbD] WhatsApp: Spam, inundaciÃ\\x83Â³n y robo de cuentas. Vector de ataque local http://t.co/BZMTSx4H ContribuciÃ\\x83Â³n gracias a ...\', \'Evasi0n needs the iPhone to be activated. Is there a way to hacktivate? @evad3rs\', "RT @Queen_Europe: Don\'t say anything but I swear I can see an envelope in his back pocket. #Rajoy", \'RT @S21sec: Actualiza Java (sÃ\\x83Â\\xad, de nuevo) http://t.co/EpFPn4Gk vÃ\\x83Â\\xada @seguin

In [123]:
import ast
import re
def clean_tweets(df):
    def process_tweets_list(corpus):
        
        corpus_processed = []
        for tweet_list in corpus:
            tweet_list = str(tweet_list)
            row_processed = ""
            
            #replace RT and @
            row_processed = tweet_list.replace("RT", "" ) 
            row_processed = row_processed.replace("@", "" )
            
            row_processed = re.sub(r'http\S+', "", row_processed) #remove any URLs in tweets
            row_processed = re.sub(r'[^\x00-\x7f]', "", row_processed) #remove Non-ASCII characters
            row_processed = re.sub(r'[^\w\s]', '', row_processed) # remove punctuation
            corpus_processed.append(row_processed if not pd.isnull(row_processed) else "") # handle NA
            

        return corpus_processed
    
    def process_description(corpus):
        corpus_processed = []
        
        return corpus_processed
    
    df["tweets_list_processed"] = process_tweets_list(df["tweets_list"])
    
    return df
            

In [124]:
clean_tweets(df_tweets)

,Unnamed: 0,Unnamed: 0.1,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,...,profile_link_color,utc_offset,protected,verified,description,created_at,updated,account_type,tweets_list,tweets_list_processed
0,0,1,22903,effeffe,effeffe,164,132,194,12,4,...,9999,3600.0,NaN,NaN,L'uomo ha creato dio a sua immagine e somiglia...,Sun Nov 26 15:19:32 +0000 2006,14/2/2015 11:32,real,['@TheFakeProject cerca followers reali!!! #Im...,TheFakeProject cerca followers reali ImNotAFak...
1,1,3,286543,Alessio Bragadini,abragad,6892,930,535,478,28,...,0000FF,3600.0,NaN,NaN,Web and social media developer from Italy,Wed Dec 27 14:55:17 +0000 2006,14/2/2015 11:32,real,"[""RT @hotdogsladies: The real problem with ema...",hotdogsladies The real problem with email isn...
2,2,4,438023,fullcaffeine,fullcaffeine,2885,173,444,41,2,...,9999,3600.0,NaN,NaN,NaN,Tue Jan 02 09:01:50 +0000 2007,14/2/2015 11:32,real,['Amare il #giornalismo : @Internazionale @la_...,Amare il giornalismo Internazionale la_stampa...
3,3,5,586003,Maurizio Tesconi,myself2048,216,97,234,2,0,...,0000FF,3600.0,NaN,NaN,CNR Researcher. Web developer. Social media ge...,Fri Jan 05 16:20:42 +0000 2007,14/2/2015 11:32,real,"['RT @TheFakeProject: Dear followers, phase 2 ...",TheFakeProject Dear followers phase 2 of TheF...
4,4,6,628563,Massimo Moretti,MaxMoretti,505,154,314,0,3,...,0084B4,3600.0,NaN,NaN,The truth is out there!,Fri Jan 12 11:06:17 +0000 2007,14/2/2015 11:32,real,['@Swype when the Italian dictionary will be ...,Swype when the Italian dictionary will be rel...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11107,11107,16273,385095166,Hull Util Jobs,tmj_UKH_UTIL,2,332,310,0,0,...,4A913C,-25200.0,NaN,NaN,Follow this account for geo-targeted Utilities...,Tue Oct 04 21:25:35 +0000 2011,2016-03-15 13:49:14,fake,NaN,nan
11108,11108,16274,385096188,Glasgow Util Jobs,tmj_UKG_UTIL,1,327,308,0,4,...,4A913C,-25200.0,NaN,NaN,Follow this account for geo-targeted Utilities...,Tue Oct 04 21:28:08 +0000 2011,2016-03-15 13:49:14,fake,NaN,nan
11109,11109,16275,398274058,Sandwich Uk Mgmt,tmj_SND_mgmt,2,276,265,0,1,...,4A913C,-25200.0,NaN,NaN,Follow this account for geo-targeted Business/...,Tue Oct 25 20:41:55 +0000 2011,2016-03-15 13:49:14,fake,NaN,nan
11110,11110,16292,631519728,PR Business/Mgmt,tmj_ptr_mgmt,36,527,278,0,21,...,4A913C,-25200.0,NaN,NaN,Follow this account for geo-targeted Business/...,Mon Jul 09 21:35:55 +0000 2012,2016-03-15 13:49:15,fake,NaN,nan


In [131]:
#wtf eh the language like not even english 
df_tweets.tweets_list_processed[4]

'Swype  when the Italian dictionary will be released Noverca prego cancellare la registrazione e tutti i miei dati sul vs sito delex num 3891190354 anche perchx83 mi avete bonificato il saldo Bip_Mobile Il nokia 100 non e UMTS ma la vostra SIM va una meraviglia Noverca come faccio a cancellare la mia registrazione sul vostro sito visto che ho disattivato la SIM Noverca mi x83 scaduta la sim ho inviato il modulo per la restituzione del credito quanto tempo ci vuole per riaverlo Bip_Mobile ho letto che vi appoggiate a TRE per lUMTS e a TIM per il GSM E vero Con un Nokia 100 la Vs SIM funziona Piazza Izzalini  Che coglioni  Auschwitz Ero il numero 220543 di Denis Avey con Rob Broombyfinito  Giapponese x cena  Reti domestiche  terza edizione di Maurizio Parrinofinito  TheFakeProject ImNotAFake vi seguirx83 e vi darx83 una mno A free and open world depends on a free and open web And a free and open web depends on me  freeandopen CassiniOfficial Ciao comico Il Doc dorme al corso CQC  DellAiu